In [1]:

import numpy as np
import pandas as pd 

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
clf = RandomForestClassifier(n_estimators = 100) 

from sklearn import metrics

def load (input):
    
    data = pd.read_csv(input, delimiter = ",")
    extraced_ssid_level_pivot =data.pivot(columns=['bssid'], values='level')
    
    output =pd.concat([data, extraced_ssid_level_pivot] , axis=1)
    if 'level' in output:
        del output['level']
    if 'ssid' in output:
        del output['ssid']
    if 'bssid' in output:
        del output['bssid']
    return output

def get_features_and_target(df , target_to_predicte):
    target=df[target_to_predicte]
    if target_to_predicte in df:
        del df[target_to_predicte]
    features=df
    return features, target

dataset  = load('donnees.txt') ;   # on souhaitera ajouter des donnees dans ce dataset
dataset['salleid'] = dataset['salleid'].astype(str)


In [2]:

dataset = dataset.groupby('positionid').agg("max").fillna(100)
dataset

,batimentid,salleid,floorid,centrefrequence0,frequency,00:4e:35:c8:ce:24,1c:28:af:b2:95:a0,1c:28:af:b2:95:a1,1c:28:af:b2:95:a2,1c:28:af:b2:95:a3,...,9c:1c:12:1d:9e:40,9c:1c:12:1d:9e:41,9c:1c:12:1d:9e:42,9c:1c:12:1d:9e:43,9c:1c:12:1d:9e:45,bc:9f:e4:8b:52:a0,bc:9f:e4:8b:52:a2,bc:9f:e4:8b:52:a3,bc:9f:e4:8b:52:a4,bc:9f:e4:8b:52:a5
positionid,,,,,,,,,,,,,,,,,,,,,
1,1,113,1.0,5580,5580,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,1,113,1.0,5745,5745,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
3,1,113,1.0,5745,5745,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,1,113,1.0,5745,5745,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
5,1,113,1.0,5745,5745,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,1,4,1.0,5745,5745,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
312,1,4,1.0,5580,5580,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
313,1,4,1.0,5580,5580,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-75.0,-74.0,-76.0,-75.0


In [3]:
for feature in dataset:
    print(feature)

batimentid
salleid
floorid
centrefrequence0
frequency
00:4e:35:c8:ce:24
1c:28:af:b2:95:a0
1c:28:af:b2:95:a1
1c:28:af:b2:95:a2
1c:28:af:b2:95:a3
1c:28:af:b2:97:e0
1c:28:af:b2:97:e1
1c:28:af:b2:97:e2
1c:28:af:b2:97:e3
1c:28:af:b2:97:e4
1c:28:af:b2:97:f0
1c:28:af:b2:97:f1
1c:28:af:b2:97:f2
1c:28:af:b2:97:f3
1c:28:af:b2:97:f4
1c:28:af:b2:9c:c0
1c:28:af:b2:9c:c1
1c:28:af:b2:9c:c2
1c:28:af:b2:9c:c3
1c:28:af:b2:9c:c4
1c:28:af:b2:9c:d0
1c:28:af:b2:9c:d1
1c:28:af:b2:9c:d2
1c:28:af:b2:9c:d3
1c:28:af:b2:9c:d4
1c:28:af:b2:b9:a0
1c:28:af:b2:b9:a1
1c:28:af:b2:b9:a2
1c:28:af:b2:b9:a3
1c:28:af:b2:b9:a4
1c:28:af:b2:c8:40
1c:28:af:b2:c8:41
1c:28:af:b2:c8:42
1c:28:af:b2:c8:43
1c:28:af:b2:c8:44
1c:28:af:b2:c8:50
1c:28:af:b2:c8:51
1c:28:af:b2:c8:52
1c:28:af:b2:c8:53
1c:28:af:b2:c8:54
1c:28:af:b2:c8:e0
1c:28:af:b2:c8:e1
1c:28:af:b2:c8:e2
1c:28:af:b2:c8:e3
1c:28:af:b2:c8:e4
1c:28:af:b2:c8:f0
1c:28:af:b2:c8:f1
1c:28:af:b2:c8:f2
1c:28:af:b2:c8:f4
1c:28:af:b2:f7:60
1c:28:af:b2:f7:61
1c:28:af:b2:f7:62
1c:28:af:b

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

def learn_and_generate_model() : 
    X, y =get_features_and_target(dataset, 'salleid')
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(X_train, np.asarray(y_train, dtype="|S6"))
    y_pred =  neigh.predict(X_test)
    # metrics are used to find accuracy or error   
    print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(np.asarray(y_test, dtype="|S6"), y_pred))
    return y_pred,y_test,X_train, y_train,neigh , confusion_matrix(np.asarray(y_test, dtype="|S6"), y_pred)

y_pred,y_test,X_train, y_train,neigh , conf = learn_and_generate_model()
print(conf)
print (y_train)
print (y_test)
print (y_pred)

ACCURACY OF THE MODEL:  0.5
[[1 1 1 0 0 1 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2]]
positionid
201    100
206    100
209    104
8      118
29     211
312      4
204    100
307      1
20     214
303      1
26     211
214    104
14     220
215    108
4      113
18     214
104      0
9      118
221    108
7      118
102      0
313      4
302      1
205    100
314      4
16     220
28     211
107      0
27     211
207    100
25     211
202    100
308      4
216    108
12     220
33     207
311      4
218    108
301      1
306      1
103      0
30     207
2      113
211    104
22     214
3      113
24     211
21     214
219    108
309      4
15     220
210    104
Name: salle

In [5]:
neigh

KNeighborsClassifier()

In [6]:
import pickle
from sklearn import tree
pickle.dump(clf, open('model.pkl','wb'))

In [7]:
#data = dataset['00:4e:35:c8:ce:24']=9999
def clean_input(dataset , X_dict): 
    X_dataframe = pd.DataFrame.from_dict(X_dict)
    for feature in X_dataframe: 
        if feature in dataset:
            print(feature)
        else:
            del X_dataframe[feature]
    return  X_dataframe

def threat_input(dataset , X_input):
    X_input= clean_input(dataset , X_input)
    key_array = []
    for feature in X_input:
        key_array.append(feature)
    values = X_input.values
    maj_dataset =  dataset.append(dict(zip(key_array, *values)), ignore_index=True)
    formated_input = maj_dataset.replace(np.nan , 100)
    formated_input = formated_input[-1:].values
    return formated_input

In [8]:
from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle as p
import json
from pandas import json_normalize

app = Flask(__name__)

@app.route('/predict', methods=['GET','POST'])
def makecalc():
    print(request.json)
    observation = threat_input(dataset , request.json)
    print(threat_input(dataset , request.json))
    print ("predicted" , neigh.predict(observation) , type(neigh.predict(observation)))
    return np.array2string(neigh.predict(observation))

@app.route('/app', methods=['GET','POST'])
def makecalcc():
    print(request.json)
    return "computed"
if __name__ == '__main__':
    modelfile = 'model.pkl'
    model = p.load(open(modelfile, 'rb'))
    app.run(debug=False, host='0.0.0.0')
# ici le serveur traite les donnees qu'il recoit 

ModuleNotFoundError: No module named 'flask'

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


dataset  = load('donnees.txt') ;   # on souhaitera ajouter des donnees dans ce dataset
dataset['salleid'] = dataset['salleid'].astype(str)

dataset = dataset.groupby('positionid').agg("max").fillna(100)
dataset

X, y =get_features_and_target(dataset, 'salleid')
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
clf = RandomForestClassifier(n_estimators=450, max_depth=4, random_state=0)
clf.fit(X_train, y_train)

y_pred =clf.predict(X_test)
y_pred


print (clf.score(X_test, y_test))

0.4230769230769231


In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)
y_te =clf.predict(X_test)
clf.score(X_test, y_test)
